### Data Loading

In this section, we configure the repository paths and load the datasets (query dataset and document corpus). We use the Hugging Face datasets library. If the data is already downloaded, it is loaded from the disk.

In [25]:
repo = 'spanish-ir/'
d = { 'dataset': 'messirve', 'corpus': 'eswiki_20240401_corpus' }

In [26]:
import os.path
import datasets

dataset, corpus = None, None

if not os.path.isdir('dataset'):
    revision = '1.2'
    country = 'full'
    dataset = datasets.load_dataset(repo + d['dataset'],
        revision=revision, country=country)

if not os.path.isdir('corpus'):
    corpus = datasets.load_dataset(repo + d['corpus'])

If the dataset and corpus were not downloaded in the previous step (because they already existed), we load them from the local disk.

In [27]:
# in case we already have the dataset, load them from disk
dataset = dataset if dataset is not None else datasets.load_from_disk('dataset')
train, test = dataset['train'].to_pandas(), dataset['test'].to_pandas()

corpus = corpus if corpus is not None else datasets.load_from_disk('corpus')
corpus = corpus['corpus'].to_pandas()

Once we have the test and train sets and the corpus, we can check their structure, especially the columns and their content.

In [4]:
train.info()

<class 'pandas.DataFrame'>
RangeIndex: 766296 entries, 0 to 766295
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               766296 non-null  int64  
 1   query            766296 non-null  str    
 2   docid            766296 non-null  str    
 3   docid_text       766296 non-null  str    
 4   docid_title      766296 non-null  str    
 5   query_date       766296 non-null  object 
 6   answer_date      766296 non-null  object 
 7   match_score      766296 non-null  float32
 8   expanded_search  766296 non-null  bool   
 9   answer_type      766296 non-null  str    
 10  id_country       766296 non-null  float64
dtypes: bool(1), float32(1), float64(1), int64(1), object(2), str(5)
memory usage: 463.5+ MB


In [5]:
train.head(3)

,id,query,docid,docid_text,docid_title,query_date,answer_date,match_score,expanded_search,answer_type,id_country
0,7397857,cuántos inning se juegan en el kickingball,1869086#17,El juego de kitball tiene 6 entradas y cada un...,Kickball,2024-04-07,2024-05-06,0.8829,False,feat_snip,976827.0
1,7397858,cómo beneficia la biodiversidad a la salud de...,16208#36,La biodiversidad es importante ya que cada esp...,Biodiversidad,2024-04-06,2024-05-09,1.0000,False,feat_snip,976828.0
2,7397861,quienes somos,3328953#1,"Wikipedia es una enciclopedia libre, políglota...",Wikipedia,2024-05-18,2024-06-24,1.0000,False,feat_snip,6328791.0


In [6]:
test.info()

<class 'pandas.DataFrame'>
RangeIndex: 174078 entries, 0 to 174077
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               174078 non-null  int64  
 1   query            174078 non-null  str    
 2   docid            174078 non-null  str    
 3   docid_text       174078 non-null  str    
 4   docid_title      174078 non-null  str    
 5   query_date       174078 non-null  object 
 6   answer_date      174078 non-null  object 
 7   match_score      174078 non-null  float32
 8   expanded_search  174078 non-null  bool   
 9   answer_type      174078 non-null  str    
 10  id_country       174078 non-null  float64
dtypes: bool(1), float32(1), float64(1), int64(1), object(2), str(5)
memory usage: 105.9+ MB


In [7]:
test.head(3)

,id,query,docid,docid_text,docid_title,query_date,answer_date,match_score,expanded_search,answer_type,id_country
0,7397859,en grecia quién aplico la democracia radical,87525#2,Efialtes es considerado por muchos historiador...,Efialtes de Atenas,2024-04-07,2024-05-06,1.0,False,feat_snip,976830.0
1,7397860,que conoces de la familia arduino,1337914#0,"Arduino es una compañía de desarrollo de ""soft...",Arduino,2024-05-20,2024-06-20,1.0,False,feat_snip,5870869.0
2,7397866,1 arroba cuantas kilogramos tiene,77666#7,En Bolivia y Perú hojas de coca se comercializ...,Arroba (unidad de masa),2024-04-09,2024-05-07,1.0,False,feat_snip,976874.0


In [8]:
corpus.info()

<class 'pandas.DataFrame'>
RangeIndex: 14047759 entries, 0 to 14047758
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   docid   str  
 1   title   str  
 2   text    str  
dtypes: str(3)
memory usage: 5.3 GB


In [9]:
corpus.head(3)

,docid,title,text
0,7#0,Andorra,"Para otros usos de este término, véase Andorra..."
1,7#1,Andorra,"Andorra, oficialmente Principado de Andorra ()..."
2,7#2,Andorra,"Con sus 468 km² de extensión territorial, Ando..."


### Initial Filtering and Cleaning

To optimize memory usage, we remove columns from the dataset that are not required for the search (dates, metadata, etc.).

In [ ]:
columns = ['id', 'docid_text', 'query_date', 'answer_date', 'expanded_search', 'answer_type', 'id_country']
train.drop(columns=columns, inplace=True)
test.drop(columns=columns,  inplace=True)

Next, the corpus is filtered to retain only the documents that appear in the training or test sets.

In [29]:
import gc

# select only the documents that are present in the dataset
docids = set(train['docid']) | set(test['docid'])
subcorpus = corpus[corpus['docid'].isin(docids)].copy()

# free memory
del corpus
gc.collect()

930

### Preprocessing

We define two preprocessing strategies depending on the retrieval method to be used:

- For ``TF-IDF``: A heavy cleaning will be applied, including stopword removal and normalization to reduce noise.

- For ``Embeddings``: A light cleaning will be performed, involving only lowercasing and punctuation removal, as semantic models require the context provided by functional words.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
stops = set(stopwords.words('spanish'))

def preprocess_tfidf(text: str) -> str:
    """ Applies strong preprocessing to a text """
    text = text.lower()
    # remove punctuation signs
    text = re.sub(r'[^\w\s]','', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # remove stopwords
    words = [w for w in text.split() if w not in stops]
    return ' '.join(words)


def preprocess_embeddings(text: str) -> str:
    """ Applies light preprocessing to a text """
    text = text.lower()
    # remove punctuation signs
    text = re.sub(r'[^\w\s]','', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    return text

Now, we use joblib to parallelize the text processing and speed up the task.

In [ ]:
from joblib import Parallel, delayed

def parallel_apply(series, f):
    # applies a function to a pandas series in parallel
    return Parallel(n_jobs=-1)(delayed(f)(x) for x in series)

Once the parallelization function is defined, we preprocess the required columns from both the test and train sets.

In [53]:
for column in ['query', 'docid_title']:
    # apply the preprocessing to the dataset columns, for embeddings
    train[column + '_raw'] = parallel_apply(train[column], preprocess_embeddings)
    test[column + '_raw'] = parallel_apply(test[column], preprocess_embeddings)

    # apply the preprocessing to the dataset columns, for tf-idf
    train[column + '_tfidf'] = parallel_apply(train[column], preprocess_tfidf)
    test[column + '_tfidf'] = parallel_apply(test[column], preprocess_tfidf)

In the same way, the corpus is processed.

In [55]:
for column in ['text', 'title']:
    # apply the preprocessing to the corpus columns for embeddings & tf-idf
    subcorpus[column + '_raw'] = parallel_apply(subcorpus[column], preprocess_embeddings)
    subcorpus[column + '_tfidf'] = parallel_apply(subcorpus[column], preprocess_tfidf)

### TF-IDF Retrieval

In this section, we will use the classic Bag of Words approach. To do this, we first vectorize the corpus and the queries, and then calculate the cosine similarity between each query and the documents.

### Vectorization of Documents and Queries

We configure the TfidfVectorizer to work with unigrams and bigrams, and we limit the vocabulary to 50,000 features to keep memory management viable.

In cases where more computational power is available, using higher-dimensional n-grams (e.g., trigrams) can help better capture textual relationships, at the cost of increased computation and memory usage.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2), 
    sublinear_tf=True, 
    max_features=50_000,
    norm='l2',
    max_df=0.85,
    min_df=2,
)

When vectorizing, we chose to combine the title and the text of the corpus to capture as much information as possible, since the title often serves as a summary.

In [ ]:
corpus_combined = subcorpus['title_tfidf'].fillna('') + ' ' + subcorpus['text_tfidf']
corpus_tfidf = vectorizer.fit_transform(corpus_combined).tocsc()

On the other hand, the train and test queries are combined to obtain all queries and vectorize them.

In [ ]:
queries = pandas.concat([train['query_tfidf'], test['query_tfidf']])
queries_tfidf = vectorizer.transform(queries).tocsc()

### Similarity Calculation and Ranking

Next, we define a function to calculate cosine similarity in batches of queries to avoid RAM allocation errors when working with very large matrices. That said, we use argpartition to efficiently find the top-k elements within each batch, sorted by score.

In [43]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def rank_batch(query_matrix, corpus_tfidf, top_k=100):
    """ Gets the top k most similar documents for each query """
    # obtain similarity (query - corpus)
    scores = cosine_similarity(
        query_matrix, 
        corpus_tfidf,
        dense_output=False)
    results = []

    for i in range(scores.shape[0]):
        # for each query, get the top k results
        # get the top k indices, sorted by score
        row = scores[i]
        k = min(top_k, row.nnz)
        top_idx = np.argpartition(row.data, -k)[-k:]
        doc_idx = row.indices[top_idx]
        doc_scores = row.data[top_idx]
        order = np.argsort(-doc_scores)
        results.append(doc_idx[order])
    return results

Once this function is defined, if a file with the already calculated rankings does not exist, they are computed and saved to disk.

In [ ]:
if not os.path.isfile('rankings_tfidf.npy'):
    batch_size = 256
    rankings_tfidf = []
    top_k = 100

    # rank the queries using tf-idf & save the results to disk
    for i in range(0, queries_tfidf.shape[0], batch_size):
        batch = queries_tfidf[i:i+batch_size]
        batch_rankings = rank_batch(batch, corpus_tfidf, top_k=top_k)
        rankings_tfidf.extend(batch_rankings)

    # save the ranking to a file
    normalized_ranking = []
    for ranking in rankings_tfidf:
        if ranking.shape[0] < top_k:
            # add padding to the ranking if len < 100
            ranking = np.pad(ranking, (0, top_k - ranking.shape[0]),
                constant_values=-1)
        normalized_ranking.append(ranking)
    np.save('rankings_tfidf.npy', np.array(normalized_ranking))
else:
    # load the tf-idf rankings from disk
    rankings_tfidf = np.load('rankings_tfidf.npy')

### Retrieval with Embeddings

Now, we use a semantic model (sentence-transformers) to convert texts into dense vectors. This method typically captures semantic meaning more effectively because, unlike models such as TF-IDF that focus on exact word matching, these can capture similarities between different words that share the same meaning.

#### Preparation

First of all, we combine the title and text of the corpus (just as in the previous section), but in this case, using the preprocessing specifically designed for embeddings.

In [34]:
corpus_combined_emb = subcorpus['title_raw'].fillna('') + ' ' + subcorpus['text_raw']

Now we must load the model that will be responsible for calculating the embeddings. Specifically, we will use [sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2). This model maps sentences and paragraphs to a 384 dimensional dense vector space.

It is true that a language-specific model for Spanish could have been used, such as [hiiamsid/sentence_similarity_spanish_es](https://huggingface.co/hiiamsid/sentence_similarity_spanish_es), or a multilingual one with more dimensions like [sentence-transformers/paraphrase-multilingual-mpnet-base-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2). However, my machine's computing capacity is standard, and such models could take several hours to process. Furthermore, this is an experimental project rather than a production-ready one.

In [35]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1271.54it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


#### Corpus Encoding

Once the model is loaded, we generate the embeddings for all documents in the corpus. If the file containing the encoded corpus already exists, it will be loaded instead of repeating the process, which can be time-consuming.

In [ ]:
if not os.path.isfile('corpus_embeddings.npy'):
    # encode the corpus using the sentence transformer model, then save to disk
    corpus_embeddings = model.encode(corpus_combined_emb.tolist(), batch_size=64, 
        show_progress_bar=True, normalize_embeddings=True)
    np.save('corpus_embeddings.npy', corpus_embeddings)
else:
    # load the corpus embeddings from disk
    corpus_embeddings = np.load('corpus_embeddings.npy')

Batches: 100%|██████████| 2943/2943 [1:02:35<00:00,  1.28s/it]


#### Query Encoding

Now, we generate the embeddings for all queries (both train and test).

In [37]:
all_queries = pandas.concat([train['query_raw'], test['query_raw']]).tolist()

In [ ]:
if not os.path.isfile('query_embeddings.npy'):
    # encode the queries using the sentence transformer model
    query_embeddings = model.encode(all_queries, batch_size=64, 
        show_progress_bar=True, normalize_embeddings=True)
    np.save('query_embeddings.npy', query_embeddings)
else:
    # load the query embeddings from disk
    query_embeddings = np.load('query_embeddings.npy')

Batches: 100%|██████████| 14694/14694 [32:43<00:00,  7.48it/s]


#### Ranking with Embeddings

Since the vectors are normalized, cosine similarity is equivalent to the dot product. This allows us to calculate similarities between queries and the corpus very quickly through matrix multiplication, again using batches instead of processing all the data at once.

In [39]:
def rank_batch_embeddings(query_emb, corpus_emb, top_k=100):
    """ Gets the top k most similar documents for each query using embeddings """
    scores = query_emb @ corpus_emb.T
    rankings = []

    for i in range(scores.shape[0]):
        # for each query, get the top k results
        # get the top k indices, sorted by score
        row = scores[i]
        part_idx = np.argpartition(row, -top_k)[-top_k:]
        top_scores = row[part_idx]
        sorted_order = np.argsort(-top_scores)
        top_k_idx = part_idx[sorted_order]
        rankings.append(top_k_idx)
    return rankings

In [40]:
if not os.path.isfile('embedding_rankings.npy'):
    batch_size = 256
    embedding_rankings = []

    # rank the queries using embeddings & save the results to disk
    for i in range(0, query_embeddings.shape[0], batch_size):
        batch = query_embeddings[i:i+batch_size]
        batch_rankings = rank_batch_embeddings(batch, corpus_embeddings)
        embedding_rankings.extend(batch_rankings)
    np.save('embedding_rankings.npy', np.array(embedding_rankings))
else:
    # load the embedding rankings from disk
    embedding_rankings = np.load('embedding_rankings.npy')

#### Mapping Indices to IDs

The current rankings contain numerical indices (positions in the subcorpus array). We need to convert them into the actual ``docid`` values to evaluate them correctly.

In [41]:
idx_to_docid = subcorpus['docid'].values

We convert both TF-IDF and embedding rankings:

In [45]:
rankings_tdidf_final = []
for query_ranking in rankings_tfidf:
    top_docids = [idx_to_docid[idx] for idx in query_ranking]
    rankings_tdidf_final.append(top_docids)

In [46]:
rankings_emb_final = []
for query_indices in embedding_rankings:
    top_docids = [idx_to_docid[idx] for idx in query_indices]
    rankings_emb_final.append(top_docids)

### Evaluation

Finally, we evaluate both systems (TF-IDF and Embeddings) using three standard Information Retrieval metrics: Precision@k, Recall@k, and nDCG@k. To do this, we define the following evaluation function that compares the ground truth document (y_true) with the list of retrieved documents (rankings).

In [47]:
import math

def evaluate(y_true, rankings, k_values=[1,5,10]):
    """ Evaluates a IR system """
    metrics_results = {k: {'precision': 0.0, 'recall': 0.0, 'ndcg': 0.0} for k in k_values}
    n_queries = len(y_true)

    for true_doc_id, predicted_docs in zip(y_true, rankings):
        # find the idx of the correct document in the list
        rank_dict = { doc: idx + 1 for idx, doc 
            in enumerate(predicted_docs) }
        rank = rank_dict.get(true_doc_id, float('inf'))

        for k in k_values:
            if rank <= k:
                k_metric = metrics_results[k]
                k_metric['precision'] += 1.0 / k
                k_metric['recall'] += 1.0
                k_metric['ndcg'] += 1.0 / math.log2(rank + 1)

    final_metrics = {}
    for k in k_values:
        k_metric = metrics_results[k]
        final_metrics[f'Precision@{k}'] = k_metric['precision'] / n_queries
        final_metrics[f'Recall@{k}'] = k_metric['recall'] / n_queries
        final_metrics[f'nDCG@{k}'] = k_metric['ndcg'] / n_queries
    return pandas.DataFrame([final_metrics])

Once the function is defined, we prepare the test data (ground truth).

In [48]:
y_true = test['docid'].tolist()
n_train = len(train)

First, we evaluate TF-IDF, using only the portion of the rankings that corresponds to the test set.

In [49]:
rankings_tfidf_test = rankings_tdidf_final[n_train:]
evaluate(y_true, rankings_tfidf_test)

,Precision@1,Recall@1,nDCG@1,Precision@5,Recall@5,nDCG@5,Precision@10,Recall@10,nDCG@10
0,0.100168,0.100168,0.100168,0.051695,0.258476,0.181666,0.034316,0.343157,0.209103


In the same way, the embeddings are evaluated.

In [50]:
rankings_emb_test = rankings_emb_final[n_train:]
evaluate(y_true, rankings_emb_test)

,Precision@1,Recall@1,nDCG@1,Precision@5,Recall@5,nDCG@5,Precision@10,Recall@10,nDCG@10
0,0.174945,0.174945,0.174945,0.069731,0.348654,0.266843,0.041863,0.418628,0.289532


### Conclusions

In this work, an information retrieval system in Spanish has been implemented and evaluated using the Messirve dataset and a subset of Wikipedia. Two approaches were compared: a lexical model based on TF-IDF and a semantic model based on embeddings.

The results obtained show a clear superiority of the embedding-based model over the TF-IDF approach across all evaluated metrics. Specifically, Precision@1 increases from 0.100 with TF-IDF to 0.175 with embeddings, representing a significant improvement in the proportion of queries where the correct document appears in the top position. Likewise, Recall@10 rises from 0.343 to 0.419, indicating that the semantic model retrieves the relevant document within the top 10 in a greater number of cases. Consistently, nDCG@10 also improves from 0.209 to 0.290, reflecting a better ranking of the results.

These findings confirm that the embedding model captures semantic similarity between queries and documents more effectively than the lexical model based on term matching. While TF-IDF relies on exact word matches, embeddings allow for generalization across synonyms and variations, providing much greater flexibility.

As potential future improvements, we propose exploring hybrid approaches that combine sparse and dense models, using more advanced embedding models, applying re-ranking techniques with cross-encoders, or employing more complex and language-specific models for Spanish, as previously discussed in this notebook.